In [1]:
import urllib.request
from nltk import re
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
import nltk
import numpy as np
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
        

In [6]:
def gen_sys(urls):
    #for loop to go parse list of URLs
    for i in urls:
        response= urllib.request.urlopen(i)
        html=response.read()
        newhtml= html.decode("utf-8")
        soup = BeautifulSoup(newhtml, 'html.parser')
        #Extacting the text content 
        addr1=str(soup.get_text())
        y=soup.get_text()
        addr1=addr1.strip()
        #cleaning the string
        str4=re.sub(r'\n+', '\n', y).strip()
        str4=str4.replace(addr1,"")
        w1 = re.sub(r'([a-z])\.([A-Z])', r'\1. \2', str4)
        tokenizer = RegexpTokenizer(r'\w+')
        tokens=tokenizer.tokenize(w1)
        t=list()
        for i in tokens:
            # removing the punctuation
            t.append(' '.join(re.sub('[^A-Za-z0-9]+', '', i)) )
        t=[i.replace(' ','') for i in t]
        t=" ".join(t)
        t=re.sub(r' [0-9]+',' ',t) 
        # Saving the HTML content
        np.savetxt(r'html.txt',list([t]),fmt='%s')
        w1.replace(r'(-)+|( -)+' ,'')
        regex = re.compile('[^a-zA-Z]')
        regex.sub('',w1)
        r=list([w1])
        r=w1.split(' ')
        regex = re.compile('[^a-zA-Z\.]')
        t1=[regex.sub('',i) for i in r]
        t1=" ".join(t1)
        docs = sent_tokenize(t1.lower())
        np.savetxt(r'sent_split.txt',docs,fmt='%s')
        tokenizer = RegexpTokenizer(r'\w+')
        tokens=tokenizer.tokenize(t.lower())
        stop_words = set(stopwords.words('english'))
        filtered_sentence = [w for w in tokens if not w in stop_words]
        np.savetxt(r'norm.txt',filtered_sentence,fmt='%s')
        np.savetxt(r'tokens.txt',tokens,fmt='%s')
        tagged = nltk.pos_tag(tokens)
        np.savetxt(r'pos_tags.txt',tagged,fmt='%s')
        def tokenize(text):
            tokens = nltk.word_tokenize(text)
            stems = []
            for item in tokens:
                stems.append(PorterStemmer().stem(item))
                stem=listToString(stems)
                np.savetxt(r'stemming.txt',stems,fmt='%s')
            return stem
        def listToString(s):  
            str1 = " " 
            return (str1.join(s)) 
        stemmed=list()
        for i in docs:
            st=tokenize(i)
            stemmed.append(st)
        # settings that you use for count vectorizer will go here
        tfidf_vectorizer=TfidfVectorizer(stop_words='english',use_idf=True)
        # just send in all your docs here
        tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)
        # get the first vector out (for the first document)
        first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
        # place tf-idf values in a pandas data frame
        df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
        df=df.sort_values(by=["tfidf"],ascending=False)
        df1=df['tfidf'].index.to_list()
        df2=df['tfidf'].values
        #Fin
        d = {'words':df1,'tfidf':df2}
        #Fin
        fin_df = pd.DataFrame(d)
        #Fin
        x1=list()
        for index, row in df.iterrows():
            if row['tfidf']>0.0:
                x1.append([index,row['tfidf']])
        np.savetxt(r'tf_idf.txt',x1,fmt='%s')
        
        


In [ ]:
#Enter list of URLs to process them
gen_sys(["http://www.multimediaeval.org/mediaeval2019/memorability/","https://sites.google.com/view/siirh2020/"])